# Introduction 

## Objective

### @todo:

- [ ] introduce "head counts" and "sweeps" for narrative purposes

- Answer the question:
  * What is the relationship between observations recorded as "head counts" and those recorded as "sweeps"?
- Document the process of finding the answer so that the exact steps are replicable and clearly demonstrated here.

You can follow along in your own Jupyter notebook, or open [this notebook on mybinder.org] and replay cells while tweaking the code if you want to experiment. You'll find [this notebook project on GitHub].

[this notebook on mybinder.org]: https://mybinder.org/v2/gh/devvyn/aafc-field-data/master?filepath=notebook%2Fprojects%2F2016-sweep-vs-tiller%2F2016-sweep-vs-tiller.ipynb
[this notebook project on GitHub]: https://github.com/devvyn/aafc-field-data/tree/master/notebook/projects/2016-sweep-vs-tiller

## About the Data

!["Perfectly straight line in graph comparing supposedly independent variables in Head Count and Sweeps workbook, in Excel Online"](head-count-sweeps-graph-excel-online.png)

I'm troubleshooting a spreadsheet document that was prepared by someone else. Whoever prepared it probably handed it off to someone else before I came into contact. The information in the document looks like it's been copied from other documents which I'm not certain I have access to.

In this document, there's a graph that demonstrates the goal of the document: to compare the relationship between two observation methods. Unfortunately, the graph shows a suspicious degree of idealness: a perfect one-to-one ratio across the entire domain.

My assignment is to trace the error and correct the graph so that it displays the precise ratios calculated from appropriate samples. The samples available for this purpose are counts of aphids collected by the two methods described in the [Objective].

[Objective]: #Objective

### Explore Worksheets

Before getting hands on with _pandas_, I'll open the document in Excel and record what my senses tell me.

#### Unbelievable Graph

- There's a graph in a worksheet called "head counts vs sweeps graphs" which demonstrates the analytical problem encountered/developed by someone else.
- The data supposedly being compared in the graph is cannot be the data that was intended for comparison because the ratio depicted is perfectly linear even though it's comparing real world samples.

#### Work Not Shown

It's clear that the Excel workbook has the results of many calculations, yet there are no spreadsheet formulas in any cells. Spreadsheet formulas would have greatly expedited the verification of the calculations; without any reference to how the calculations were performed and on what values, I'll need to replicate them from scratch.

#### Metadata and Document History

- Data may have been copied from multiple, unidentified sources.
- It's not clear which data is "original" and which is duplicated, amongst the worksheets in the workbook.
- Some data appears to have been summed and then mixed back in with the rest of the data.
- There are no notes from editors of the workbook, and the editors are unidentifiable.

### Summary of Issues

- No metadata
- No convention for categorical label values
- Mixed aggregation levels (some sums mixed with non-summed data)
- Graph plot is based on incorrect values

## Analytical Goals

### Tidy & Plot Ratios

- Data is clean and indexed well enough to align quantitative samples from the two field collection methods.
- Upon plotting the ratios of dependent variables corresponding with each collection method, the graph should not depict an absolutely perfect fit to a line. That is to say, there should be some variation.

### Retain for Reference

- For posterity, the data should be easy to align with any similar data. This makes it possible to identify whether the same data exists in any other file.

## Document Conventions

I'll use certain formatting conventions for written terms throughout this document.

### Technical Terms

In cases where a technical term also has a common, non-technical meaning, the text will be shown in *italics*.

>*pandas* is a data processing framework for Python which uses readable yet expressive syntax to perform calculations upon many values or many sets of values, such as combining spreadsheets or even plotting visualizations.

### Worksheet and DataFrame

This document deals with data sets from Excel spreadsheet workbooks, each of which are handled as a *pandas* `DataFrame`. These will be shown in **bold**, including Python variables referring to a `DataFrame`.

>I've imported **Sheet2** into *pandas* and temporarily assigned it to the variable **s2** for convenience.

### Column and Series

Each column from the workbooks becomes a `Series` in *pandas*. These will be shown in ***bold italics***, including Python variables referring to a `Series`.

>Whenever ***number of samples*** is not a number, ***distance(m)*** is always equal to "`Combined`".

### Code Text and Verbatim Expression

Whenever a word is meant to refer to an expression used by *pandas* or Python, it will be shown in `monospace`. The given expression must always be typed exactly as shown when using it in code execution. This applies equally whether the text is used as an executable expression (Python command) or just a quoted snippet of text used as a non-executable value (eg index label).

>The `concat` command in *pandas* combines two or more `DataFrame` objects.

# Procedure

## Setup Computing Environment

### Interactive Document

Document handling and analysis will be conducted in [Jupyter] Notebook/Lab, using [Python] 3. To setup a live workspace, consult the [README] for the parent project comprising this document and its associated files.

[Jupyter]: https://jupyter.org/
[Python]: https://www.python.org/about/
[README]: https://github.com/devvyn/aafc-field-data/blob/master/README.md

### Source Data

To follow along, you'll require direct access to [`2016-sweep-vs-tiller.xlsx`][file] within your workspace. Download the [file] from GitHub if you're not cloning the entire source [repository].

If you're interacting on [mybinder.org], the file should already be present.

[repository]: https://github.com/devvyn/aafc-field-data/
[file]: https://github.com/devvyn/aafc-field-data/blob/master/notebook/projects/2016-sweep-vs-tiller/2016-sweep-vs-tiller.xlsx
[mybinder.org]: https://mybinder.org/v2/gh/devvyn/aafc-field-data/master?filepath=notebook%2Fprojects%2F2016-sweep-vs-tiller%2F2016-sweep-vs-tiller.ipynb
[this notebook project on GitHub]: https://github.com/devvyn/aafc-field-data/tree/master/notebook/projects/2016-sweep-vs-tiller

### Install Third Party Packages

**NOTE FOR CLOUD USERS**: Keep in mind, some notebooks in the cloud run in temporary "containers" or otherwise ethereal environments where installed packages may vanish after a certain amount of time, so you may have to install again in subsequent sessions.


#### _pandas_

* If your live workspace doesn't include it, [install _pandas_] before continuing.
* If you're using [Anaconda], you probably already have _pandas_.

If you're not sure if it's installed, you may run this command in a code cell within your notebook, prefixed by an exclamation mark as shown. The Python package installer will not install anything you already have.

```
!pip install pandas
```

[Anaconda]: http://docs.continuum.io/anaconda/
[install _pandas_]: https://pandas.pydata.org/pandas-docs/stable/install.html

#### Markdown (Optional) 

If you're following along, I recommend you install the `markdown` package for Python. It makes formatted output from code much simpler to express. You can omit this as long as you don't use the `markdown` function in your code. I'll be using it, however, so if you run my examples, remove the calls to `markdown` and `HTML` and just use plain text.

```
!pip install markdown
```

### Import Python Packages

From the standard Python library:

* `re` helps with pattern matching in text.

Third party libraries: 

* *pandas* does the heavy lifting of working with spreadsheet-like data.
* For outputting formatted text from the code and displaying in the notebook: `markdown.markdown` and `IPython.display.HTML`.

In [1]:
import re

import pandas
from markdown import markdown
from IPython.display import HTML

## Load Worksheets into _pandas_

I'll make a dictionary of names and data frames, so I can examine the file overall.

In [2]:
data_file = pandas.ExcelFile('2016-sweep-vs-tiller.xlsx')
sheets = {
    sheet_name: data_file.parse(sheet_name)
    for sheet_name in data_file.sheet_names
}
sheet_names = sheets.keys()
sheet_frames = sheets.values()

### Compare Columns

To begin to discern which data is original, I'll list the first eight columns of each worksheet, which will offer some descriptive terms for the data in each:

In [3]:
pandas.DataFrame(
    data=[frame.columns for frame in sheet_frames],
    index=pandas.Index(data=sheet_names),
).loc[:, :8]

,0,1,2,3,4,5,6,7,8
Head Counts,Site,Crop,Date,Field,Zadoks_stage,Tiller,EGA_head,EGA_leaf,BCO_head
Sheet2,ID,Province,Collection_Date,Sample_by_week,Date_by_week,Date,Julian_date,Site,Field_name
Sweep Samples Cereals,ID,Province,Collection_Date,Sample_by_week,Date_by_week,Date,Julian_date,Site,Field_name
Head Counts Edited,Date,Site,Crop,Field,Sample Type,Unnamed: 5,Zadoks_stage,Tiller,EGA_alate
Sweep Samples Cereals Edited,Date,Site,Crop,Field_name,Sample Type,Total Sweeps,Unnamed: 6,Unnamed: 7,EGA_alate
Data Sheets Combined,ID,Date,Site,Crop,Field_name,Sample Type,Total Sweeps,Zadoks_stage,Tiller
Pivot chart LH phen,None,None,None,None,None,None,None,None,None
leafhoppers 2016 cereal sweeps,Collection_Date,Sample_by_week,Date_by_week,Date,Julian_date,Site,Field_name,Crop,Distance(m)
Sheet3,None,None,None,None,None,None,None,None,None
aphid sweep vs head count,ID,Date,Site,Crop,Field_name,Sample Type,Total Sweeps,Zadoks_stage,Tiller


### Classify Worksheets

Because I can't trust the accuracy of the data used by the plot in Excel, I need to look at all the sheets and determine the most complete and unadulterated data sets. I'll determine which data belongs to each category, and compare the sets.

Based on sheet names, column names, and similarities between columns sets, I can probably group the sheets like so:

#### Tiller Head Count

- **Head Counts**
- **Head Counts Edited**

#### Net Sweep

- **Sheet2**
- **Sweep Samples Cereals**
- **Sweep Samples Cereals Edited**
- **leafhoppers 2016 cereal sweeps**

#### Extraneous

- United:
  - **Data Sheets Combined**
- Analytical experiments:
  - **Pivot chart LH phen**
  - **Sheet3**
  - **aphid sweep vs head count**
  - **head counts vs sweeps graphs**

## Tidy Up

### Normalize Column Names

I've already noticed at least one leading space on a column name ("spiders"), and variations in capitalization. To compensate for this, I'll strip all leading and trailing whitespace from the column names.

To avoid confusion due to variations in capitalization, I'll convert all column names to lower case.

In [4]:
for sheet in sheets.values():
    if sheet.columns.size > 0:
        sheet.columns = sheet.columns.str.strip().str.lower()

### Convert Date & Time Format

Before I can easily examine dates from the worksheets, I must convert them to proper date values for Python and *pandas*. I'll check the date formats:

In [5]:
(
    pandas.concat(
        {
            name: sheet.loc[:, sheet.columns.str.contains('date')].reset_index(drop=True)
            for (name, sheet) in sheets.items() if sheet.columns.size > 0
        },
        axis='columns',
    )
    .loc[0]
    .unstack()
    .fillna('')
)

,collection_date,date,date_by_week,julian_date
Data Sheets Combined,,2016-08-12 00:00:00,,
Head Counts,,04/08/2016,,
Head Counts Edited,,2016-08-04 00:00:00,,
Sheet2,12_08_2016,Aug_12,0,0
Sweep Samples Cereals,12_08_2016,Aug_12,0,0
Sweep Samples Cereals Edited,,2016-08-12 00:00:00,,
aphid sweep vs head count,,2016-08-04 00:00:00,,
leafhoppers 2016 cereal sweeps,12_08_2016,Aug_12,0,0


According to the documentation for [`pandas.to_datetime`], I can convert a string to a true datetime value. The function uses [Python datetime format strings]. Here are the date columns, the format strings, and the worksheet names I see:

***collection_date*** (`%m_%d_%Y`):

- **leafhoppers 2016 cereal sweeps**
- **Sheet2**
- **Sweep Samples Cereals**

***date*** (`%m/%d/%Y`):

- **Head Counts**

***date*** (no conversion):

- **aphid sweep vs head count**
- **Sweep Samples Cereals Edited**
- **Head Counts Edited**
- **Data Sheets Combined**

In some worksheets, the ***date*** column is present even though there's a better column for the date (***collection_date***). I'll drop the ***date*** column and rename the ***collection_date*** column to fill its place.

After conversion, I'll output a sample date from the column to prove that the column ***date*** is in fact a datetime.

[`pandas.to_datetime`]: http://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.to_datetime.html
[Python datetime format strings]: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [6]:
for name, sheet in ((name, sheets[name]) for name in (
    'Head Counts',
    'Sheet2',
    'Sweep Samples Cereals',
    'leafhoppers 2016 cereal sweeps',
)):
    display(HTML(markdown(f'#### {name}')))
    if name == 'Head Counts':
        sheet['date'] = pandas.to_datetime(
            sheet['date'],
            format='%d/%m/%Y',
        )
    else:
        sheet.drop(columns='date', inplace=True)
        sheet.rename(
            columns={'collection_date': 'date'},
            inplace=True,
        )
        sheet['date'] = pandas.to_datetime(
            sheet['date'],
            format='%d_%m_%Y',
        )
    display(sheet.date.head(1))

0   2016-08-04
Name: date, dtype: datetime64[ns]

0   2016-08-12
Name: date, dtype: datetime64[ns]

0   2016-08-12
Name: date, dtype: datetime64[ns]

0   2016-08-12
Name: date, dtype: datetime64[ns]

## Select Primary Data Set: Sweep Samples

### Candidates

- **Sheet2**
- **Sweep Samples Cereals**
- **Sweep Samples Cereals Edited**
- **leafhoppers 2016 cereal sweeps**

From my hands-on exploration in Excel, I got the feeling that **Sheet2** is the most complete. It generally had more rows and looked tidy. I'll compare it to the other candidates, one at a time.

### **Sweep Samples Cereals** vs **Sheet2**

I'd like to confirm my hope that all data in **Sweep Samples Cereals** is included in the vaguely named **Sheet2**. I'll put them both in a dictionary called **compare_sheets** for quick reference.

In [7]:
sheet_names = (
    'Sweep Samples Cereals',
    'Sheet2',
)
compare_sheets = {
    name: sheets[name]
    for name in sheet_names
}

I'll refer to **Sweep Sample Cereals** as **ssc** and I'll continue to refer to **Sheet2** as **s2**. 

In [8]:
ssc, s2 = compare_sheets.values()

#### Columns

I'll convert each column list to a `Series` and display them side-by-side in a `DataFrame`.

In [9]:
pandas.DataFrame(
    data={
        name: sheet.columns.to_series(name=name)
        for name, sheet in compare_sheets.items()
    }
).isna().replace(to_replace={True: '×', False: '✅'})

,Sweep Samples Cereals,Sheet2
( damsel bug)nabis_americoferus_adult,✅,✅
1st_instar_ega,✅,✅
1st_instar_macrosteles,✅,✅
2nd_instar_ega,✅,✅
2nd_instar_macrosteles,✅,✅
3rd_instar_ega,✅,✅
3rd_instar_ega_pre-alate,✅,✅
3rd_instar_macrosteles,✅,✅
4th_instar_macrosteles,✅,✅
4th_instar_pre-alate,✅,✅


I see only a few differences. I'll determine the exact asymmetry using *pandas* `Index.symmetric_difference`, which is available on the `columns` property, since columns are technically also an `Index` within *pandas*:

In [10]:
ssc.columns.symmetric_difference(s2.columns).tolist()

['ega/20 sweeps',
 'ega/sweep',
 'total ega',
 'total sweeps',
 'unnamed: 129',
 'unnamed: 133']

I believe the unnamed columns are empty. The other columns have calculated values that I don't trust. **Sweep Samples Cereals** is probably useless for my purposes.

#### Date Range

In [11]:
def len_unique(pandas_object):
    return len(pandas_object.unique())


pandas.DataFrame(
    {
        name: frame.date
        for name, frame in compare_sheets.items()
    }
).apply(
    [
        pandas.Series.max,
        pandas.Series.min,
        len_unique,
    ]
)

,Sweep Samples Cereals,Sheet2
max,2017-07-08 00:00:00,2017-07-08 00:00:00
min,2016-06-05 00:00:00,2016-06-05 00:00:00
len_unique,33,34


I need to check if *all* the dates in **Sweep Samples Cereals** are in **Sheet2**.

In [12]:
ssc.date.isin(s2.date).all()

True

I'm curious if the converse is also true:

In [13]:
s2.date.isin(ssc.date).all()

False

Since **s2** has some dates which **ssc** does not, I'd like to know how big the difference is. I'll view all date values in **s2** which fail to match with those in **ssc**, ignoring duplicates.

In [14]:
(
    s2.date.loc[~s2.date.isin(ssc.date)]
    .drop_duplicates()
)

486   2017-06-17
568   2016-06-06
Name: date, dtype: datetime64[ns]

So, this isn't a lot, but it's worth noting.

#### Aggregation

I see a ***total sweeps*** column only in **Sweep Samples Cereals**, which leads me to suspect that **Sweep Samples Cereals** comprises aggregates (sums), possibly based on values present in **Sheet2**. 

However, **Sheet2** has two additional dates, which means that it has data that the other worksheet doesn't. Now, it's a matter of determining whether **Sheet2** is comprehensive enough to preclude the use of **Sweep Samples Cereals**.

#### Value Counts on ***distance(m)***

It's my impression that the actual, physical sweeps were at different distances from some physical origin, and later reduced to sums. If I peek at the ***distance(m)*** column, I should see a clear difference between these varied distances and the "distance" value used by aggregates. Ideally, there wouldn't be a distance value for aggregates at all, but this touches on the problem of mixing raw data with partially analysed data.

In [15]:
display(
    pandas.concat(
        {
            name: frame['distance(m)']
            for name, frame in compare_sheets.items()
        },
        axis='columns',
    )
    .apply(pandas.value_counts)
    .fillna('-')
)

,Sheet2,Sweep Samples Cereals
5,105,-
10,107,-
25,106,-
50,109,-
100,93,-
Combined,92,92
0,56,-


Indeed, **Sheet2** has data from observations at various "distances", while **Sweep Samples Cereals** has only the label, ***Combined***.

**Sheet2** has more rows because it isn't totalling up the sweeps from various distances. That makes **Sheet2** less processed, and therefore more "raw" (less likely to be corrupted by human error during prior analysis).
Since I don't want reduced (summed) data, I don't want **Sweep Samples Cereals**.

#### Conclusion

Optimal candidate:

- **Sheet2**

### **Sweep Samples Cereals Edited** vs **Sheet2**

In [16]:
sheets_to_compare = {
    name: sheets[name]
    for name in (
        'Sweep Samples Cereals Edited', 
        'Sheet2',
    )
}
ssce, s2 = sheets_to_compare.values()
sheet_names = sheets_to_compare.keys()

#### Columns

In [17]:
pandas.options.display.max_rows = 140
pandas.DataFrame(
    data={
        name: sheet.columns.to_series(name=name)
        for name, sheet in sheets_to_compare.items()
    }
).isna().replace(to_replace={True: '×', False: '✅'})

,Sweep Samples Cereals Edited,Sheet2
( damsel bug)nabis_americoferus_adult,✅,✅
1st_instar_ega,×,✅
1st_instar_macrosteles,×,✅
2nd_instar_ega,×,✅
2nd_instar_macrosteles,×,✅
3rd_instar_ega,×,✅
3rd_instar_ega_pre-alate,×,✅
3rd_instar_macrosteles,×,✅
4th_instar_macrosteles,×,✅
4th_instar_pre-alate,×,✅


**Sweep Samples Cereals Edited** seems to have left out some columns that would be expected to carry finely categorized subjects, such as various instars of aphids and leafhoppers. This makes it less likely to have information I'll need, because I'll be comparing aphid numbers. Furthermore, it appears that **Sweep Samples Cereals Edited** has a column, ***Total Sweeps***, that's probably an artefact from pre-existing aggregation. It's also missing the ***Distance(m)*** column; another sign of aggregation, and therefore loss of some information.

Unless further analysis reveals that **Sweep Samples Cereals Edited** has dates that are missing from **Sheet2**, I'll assume it's not worth closer examination.

#### Date Range

In [18]:
def len_unique(pandas_object):
    return len(pandas_object.unique())


pandas.DataFrame(
    {
        name: frame.date
        for name, frame in sheets_to_compare.items()
    }
).apply([
    pandas.Series.max,
    pandas.Series.min,
    len_unique,
    len,
])

,Sweep Samples Cereals Edited,Sheet2
max,2016-08-15 00:00:00,2017-07-08 00:00:00
min,2016-06-05 00:00:00,2016-06-05 00:00:00
len_unique,32,34
len,668,668


It appears that **Sweep Samples Cereals Edited**, like **Sweep Samples Cereals**, has slightly fewer unique dates that **Sheet2** but the same range, so I probably won't miss anything if I ignore it. To be sure, I need to check if all those specific dates in **Sweep Samples Cereals Edited** are actually in **Sheet2**, and not just that the range matches.

In [19]:
ssce.date.isin(s2.date).all()

True

Excellent. I see no reason to pay attention to **Sweep Samples Cereals Edited** or **Sweep Samples Cereals** anymore, except for the possibility that the actual sample data differs. For the time being, I'll ignore that possibility and move ahead. 

#### Conclusion

Optimal candidate:

- **Sheet2**

### **leafhoppers 2016 cereal sweeps** vs **Sheet2**

In [20]:
sheets_to_compare = {
    name: sheets[name]
    for name in (
        'leafhoppers 2016 cereal sweeps',
        'Sheet2',
    )
}
lh2016, s2 = sheets_to_compare.values()
sheet_names = sheets_to_compare.keys()

#### Columns

In [21]:
pandas.DataFrame(
    data={
        name: sheet.columns.to_series(name=name)
        for name, sheet in sheets_to_compare.items()
    }
).isna().replace(to_replace={True: '×', False: '✅'})

,leafhoppers 2016 cereal sweeps,Sheet2
( damsel bug)nabis_americoferus_adult,×,✅
1st_instar_ega,×,✅
1st_instar_macrosteles,✅,✅
2nd_instar_ega,×,✅
2nd_instar_macrosteles,✅,✅
3rd_instar_ega,×,✅
3rd_instar_ega_pre-alate,×,✅
3rd_instar_macrosteles,✅,✅
4th_instar_macrosteles,✅,✅
4th_instar_pre-alate,×,✅


Clearly, **leafhoppers 2016 cereal sweeps** is focused on leafhoppers. Because the object of our analysis is to compare aphid numbers, I don't see the relevance of this leafhopper data.

#### Conclusion

Given that the leafhopper data isn't pertinent, **Sheet2** remains the best candidate.

### Overall Best Data Source: **Sheet2**

After all analysis, **Sheet2** appears to be the purest, most relevant base for comparison of "sweep" sample data to that of "tiller" count data.

## Select Primary Data Set: Tiller Head

Only two worksheets in the source Excel workbook seem relevant to my need for "tiller head" samples:

- **Head Counts**
- **Head Counts Edited**

### **Head Counts** vs **Head Counts Edited**

In [22]:
sheets_to_compare = {
    name: sheets[name]
    for name in (
        'Head Counts',
        'Head Counts Edited',
    )
}
hc, hce = sheets_to_compare.values()
sheet_names = sheets_to_compare.keys()

#### Columns

In [23]:
pandas.DataFrame(
    data={
        name: sheet.columns.to_series(name=name)
        for name, sheet in sheets_to_compare.items()
    }
).isna().replace(to_replace={True: '×', False: '✅'})

,Head Counts,Head Counts Edited
aphid_mummies_blk,✅,✅
aphid_mummies_brown,✅,✅
aphids_total,✅,×
bco/head,×,✅
bco_alate,✅,×
bco_apt,✅,×
bco_head,✅,×
bco_leaf,✅,×
bco_total,✅,✅
comments,✅,✅


The main difference here seems to be the aggregation in the "edited" sheet, indicated by the columns named ***EGA/head***, ***EGA_total***, etc. For the purpose of achieving the project objective, the columns representing sums of values from other columns should be redundant because I intend to use the original values only.

#### Date Range

Now, compare for completeness:

In [24]:
def len_unique(pandas_object):
    return len(pandas_object.unique())


pandas.DataFrame(
    {
        name: frame.date
        for name, frame in sheets_to_compare.items()
    }
).apply([
    pandas.Series.max,
    pandas.Series.min,
    len_unique,
    len,
])

,Head Counts,Head Counts Edited
max,2016-08-15 00:00:00,2016-08-15 00:00:00
min,2016-07-11 00:00:00,2016-07-11 00:00:00
len_unique,20,20
len,216,216


These may be using the exact same set of dates. I'll compare:

In [25]:
hc.date.index.isin(hce.date.index).all()

True

Identical date & time for the index of each, so no basis for choosing one over the other within the date column.

### Overall Best Data Source: **Head Counts**

Since the only difference between these two frames is the unwanted extra columns, the simplest frame wins this. As mentioned earlier, that's:

- **Head Counts**

## Align Selected Data Sets

The names and corresponding data frames, from the worksheets in the source workbook document (Excel):

In [26]:
sheets_to_compare = {
    'Head Counts': hc,
    'Sheet2': s2,
}
sheet_names = sheets_to_compare.keys()
hc, s2 = sheets_to_compare.values()

### Compare Columns

Comparing alphabetically sorted column names between our two primary data sets:

In [27]:
pandas.DataFrame(
    data={
        name: sheet.columns.to_series()
        for name, sheet in sheets_to_compare.items()
    }
).isna().replace(to_replace={True: '×', False: '✅'})

,Head Counts,Sheet2
( damsel bug)nabis_americoferus_adult,×,✅
1st_instar_ega,×,✅
1st_instar_macrosteles,×,✅
2nd_instar_ega,×,✅
2nd_instar_macrosteles,×,✅
3rd_instar_ega,×,✅
3rd_instar_ega_pre-alate,×,✅
3rd_instar_macrosteles,×,✅
4th_instar_macrosteles,×,✅
4th_instar_pre-alate,×,✅


Wow, that's quite a large difference in columns for these sets. Some column names are similar to ones in the opposite `DataFrame`, so I'll need to normalize the names if I expect to align the corresponding sample values. 

Because my target sample set is only aphids, I can first drop the unimportant columns, then normalize all remaining labels.

#### Identify Columns With Aphid Sample Data

The first step in normalizing column names for aphid samples is identifying the set of columns that mention aphids by name. For better comparison, I'll filter for columns referring to aphids.

Here are some terms I know to be indicative of a relationship to aphid samples:

* aphid(s)
* ega
* bco
* greenbug

Here are equivalent *[regular expressions]* for these terms:

[regular expressions]: https://docs.python.org/3/howto/regex.html#regex-howto

In [28]:
aphid_terms = (
    r'aphids?',
#     'pea',
    'ega',
    'bco',
    'greenbug',
)

##### @todo:

- [ ] remove work related to using 'aphid' as a pattern

I'll combine all the aphid expressions into one expression that checks for all possible matches.

In [29]:
def conjoin_patterns(patterns):
    return '|'.join(patterns)

In [30]:
aphid_term_pattern = conjoin_patterns(aphid_terms)
display(aphid_term_pattern)

'aphids?|ega|bco|greenbug'

Then I'll run that check on every column name in the frames:

In [31]:
def filter_column_names(frame: pandas.DataFrame, pattern_to_match: str):
   return (
       frame
       .columns[
           frame
           .columns
           .str.contains(pattern_to_match)
       ] 
   )

In [32]:
def compare_column_names(frames_dict: pandas.DataFrame, pattern_to_match: str = '.*') -> pandas.DataFrame:
    return (
        pandas.concat(
            {
                name: filter_column_names(
                    frame,
                    pattern_to_match,
                ).to_series()
                for name, frame in frames_dict.items()
            },
            keys=sheet_names,
            axis='columns',
            sort=True,
        )
    )

In [33]:
def visualize_boolean(frame: pandas.DataFrame) -> pandas.DataFrame:
    return (
        frame
        .isna()
        .replace({
            False: '✅',
            True: '✖️',
        })
    )

In [34]:
def show_matches_in_index(
    frame:pandas.DataFrame,
    pattern_to_match:str,
    level:str = None,
    name:str='matched term'
) -> pandas.DataFrame:
    return (
        frame
        .reset_index(level)
        .rename(
            columns={
                (level or 'index'): name
            }
        )
        .replace(
            to_replace={
                name: {
                    f'({pattern_to_match})': r'🚩\1 ',
                }
            },
            regex=True,
        )
        .set_index(name)
    )

In [35]:
show_matches_in_index(
    visualize_boolean(
        compare_column_names(
            sheets_to_compare,
            aphid_term_pattern,
        )
    ),
    aphid_term_pattern,
    name='column',
)

,Head Counts,Sheet2
column,,
1st_instar_🚩ega,✖️,✅
2nd_instar_🚩ega,✖️,✅
3rd_instar_🚩ega,✖️,✅
3rd_instar_🚩ega _pre-alate,✖️,✅
🚩aphid _mummies,✖️,✅
🚩aphid _mummies_aphelinus_black,✖️,✅
🚩aphid _mummies_🚩aphid ius_brown,✖️,✅
🚩aphid _mummies_blk,✅,✖️
🚩aphid _mummies_brown,✅,✖️


I foresee some problems if I were to use these columns.

Not aphid related:

- aphidencyrtus_sp
- aphidiius_sp

Categorized as natural enemy:

- aphid_mummies
- aphid_mummies_aphelinus_black
- aphid_mummies_aphidius_brown
- aphid_mummies_blk
- aphid_mummies_brown

Analytical artifacts:

- aphids_total
- bco_total
- ega_total
- total_alate_aphids
- total_apterous_aphids

Excluding "total" and "mummies" would eliminate most of the false matches. I'll list the patterns for those, and because I'm comfortable with regular expressions, I'll go ahead and make the pattern for "mummies" work for plural and singular forms.

In [36]:
excluded_terms = (
    r'mumm(?:y|ies)',
    r'total',
)

I'll update `show_matches_in_index` to use a dictionary of patterns and corresponding marker strings (or characters).

In [37]:
def show_matches_in_index(
    frame:pandas.DataFrame,
    pattern_dict:dict,
    level:str = None,
    name:str='matched term',
) -> pandas.DataFrame:
        return (
            frame
            .reset_index(level)
            .rename(columns={(level or 'index'): name})
            .replace(
                to_replace={
                    name: {
                        f'({pattern_to_match})': f'{marker}\\1 '
                        for pattern_to_match, marker in pattern_dict.items()
                    }
                },
                regex=True,
            )
            .set_index(name)
        )

I can prevent the matching of words containing "aphid" by adding a word boundary definition:

In [38]:
def wrap_word_boundary(pattern, capture=False):
    return ''.join(
        (
            r'(?:_|^|\b)',
            '(?:' if not capture else '(?P<match>', pattern, ')',
#             r'(?:', pattern, r')',
            r'(?:_|$|\b)',
        )
    )

In [39]:
(
    aphid_term_pattern,
    excluded_term_pattern,
) = (
    wrap_word_boundary(conjoin_patterns(aphid_terms)),
    conjoin_patterns(excluded_terms)
)

In [40]:
show_matches_in_index(
    visualize_boolean(
        compare_column_names(
            sheets_to_compare,
            aphid_term_pattern,
        ),
    ),
    {
        aphid_term_pattern: '🚩',
        excluded_term_pattern: '️✖️',
    },
    name='column name',
)

,Head Counts,Sheet2
column name,,
1st_instar🚩_ega,✖️,✅
2nd_instar🚩_ega,✖️,✅
3rd_instar🚩_ega,✖️,✅
3rd_instar🚩_ega_ pre-alate,✖️,✅
🚩aphid_ ️✖️mummies,✖️,✅
🚩aphid_ ️✖️mummies _aphelinus_black,✖️,✅
🚩aphid_ ️✖️mummies _aphidius_brown,✖️,✅
🚩aphid_ ️✖️mummies _blk,✅,✖️
🚩aphid_ ️✖️mummies _brown,✅,✖️


That's much better! Now that the positive match terms and the negative match terms have been suitably processed into regular expression patterns, I can soon expect to produce the complete list of column names.

I'll use the `~` (not) operator and the `&` (and) operator of *pandas* to filter all columns appropriately:

In [41]:
def filter_column_names(
    frame: pandas.DataFrame,
    pattern_to_match: str,
    pattern_to_exclude: str = None,
) -> pandas.DataFrame:
   return (
       frame
       .columns[
           frame
           .columns
           .str.contains(pattern_to_match)
           &
           ~ (
               frame
               .columns
               .str.contains(pattern_to_exclude)
           )
       ] 
   )

If I want to use `compare_column_names` to help me visualize the effectiveness of the filter, I'll have to update that function to take the excluded terms pattern as an argument and pass it to the `filter_column_names` function which it uses.

In [42]:
def compare_column_names(
    frames_dict: pandas.DataFrame,
    pattern_to_match: str = '.*',
    pattern_to_exclude: str = None,
) -> pandas.DataFrame:
    return (
        pandas.concat(
            {
                name: filter_column_names(
                    frame,
                    pattern_to_match,
                    pattern_to_exclude,
                ).to_series()
                for name, frame in frames_dict.items()
            },
            keys=sheet_names,
            axis='columns',
            sort=True,
        )
    )

In [43]:
visualize_boolean(
    compare_column_names(
        sheets_to_compare,
        aphid_term_pattern,
        excluded_term_pattern,
    )
)

,Head Counts,Sheet2
1st_instar_ega,✖️,✅
2nd_instar_ega,✖️,✅
3rd_instar_ega,✖️,✅
3rd_instar_ega_pre-alate,✖️,✅
bco_alate,✅,✖️
bco_apt,✅,✖️
bco_head,✅,✖️
bco_leaf,✅,✖️
bird_cherry_oat_aphid,✖️,✅
ega alate,✖️,✅


Great! Aphid columns identified and it's totally clear which columns are in which `DataFrame`.

#### Normalize Aphid Names

Before combining the data sets for mathematical processing, I'll need to normalize those column names.

Although this code is probably difficult to comprehend, it will group the column names according to which term matched and which worksheet it came from. I'll be looking for aphids that were measured in both sources.

In [44]:
(
    pandas.concat(
        (
            {
                name: (
                    frame
                    .columns[
                        frame
                        .columns
                        .str.contains(aphid_term_pattern)
                        &
                        ~ (
                            frame
                            .columns
                            .str.contains(excluded_term_pattern)
                        )
                    ]
                ).to_series()
                for name, frame in sheets_to_compare.items()
            }
        ),
        sort=True,
        names=['source', 'column name'],
    )
    .str.extract(
        wrap_word_boundary(conjoin_patterns(aphid_terms), capture=True)
    )
    .reset_index(level=-1)
    .set_index('match', append=True)
    .reorder_levels(order=(1, 0), axis='index')
    .sort_index(level=(0, 1))
)

column name
match    source                                           
aphid    Sheet2                      bird_cherry_oat_aphid
aphids   Sheet2                                 pea aphids
bco      Head Counts                              bco_head
         Head Counts                              bco_leaf
         Head Counts                             bco_alate
         Head Counts                               bco_apt
ega      Head Counts                              ega_head
         Head Counts                              ega_leaf
         Head Counts                               ega_grn
         Head Counts                               ega_red
         Head Counts                             ega_alate
         Head Counts                               ega_apt
         Sheet2       sitobion_avenae_ega_green (wingless)
         Sheet2                    sitobion_avenae_ega_red
         Sheet2                                  ega alate
         Sheet2                             3rd_instar_ega
         Sheet2                   3rd_instar_ega_pre-alate
         Sheet2                             2nd_instar_ega
         Sheet2                             1st_instar_ega
greenbug Head Counts                        greenbug_alate
         Head Counts                          greenbug_apt
         Sheet2                             greenbug_aphid

##### @todo: 
- [ ] apply normalized aphid names

#### Other Columns

What non-aphid columns remain?

In [45]:
hc_remainder, s2_remainder = (
    frame.columns[~frame.columns.isin(frame_aphid)]
    for frame, frame_aphid in zip((hc, s2), (hc_aphid_columns, s2_aphid_columns))
)

sorted(hc_remainder.tolist() + s2_remainder.tolist())

NameError: name 'hc_aphid_columns' is not defined

##### @todo:

- [ ] drop other columns

#### Lookup Columns (Record Index)

Reading through the list of remaining, non-aphid related columns, I see some that don't mention any organism by name. These columns may be useful for indexing, which is crucial to aligning the two data sources.

In [ ]:
non_organism_column_names = pandas.Series(data=(
    'collection_date',
    'comments',
    'crop',
    'date',
    'date_by_week',
    'distance(m)',
    'field',
    'field_name',
    'id',
    'julian_date',
    'number of samples',
    'province',
    'sample_by_week',
    'site',
    'zadoks_stage',
))

Here are the names of the matching columns from each frame:

In [ ]:
non_organism_columns_common = pandas.DataFrame(
    {
        name: dict(zip(non_organism_column_names,
                       non_organism_column_names.isin(frame.columns)))
        for name, frame
        in zip(sheet_names, (hc, s2))
    }
).replace(to_replace={False: '×', True: '✅'})
non_organism_columns_common

### Align Columns

Our next goal is to determine which columns are in common and ensure they're of the same data type, so we can concatenate the frames.

In [ ]:
non_organism_columns_common.index[
    non_organism_columns_common.all(axis='columns')
].tolist()

Those columns alone are probably enough to align the data.

I'll need to clean up `crop` and `site`, but `date` and `collection_date` are already well formed.

These columns warrant examination as well:

- collection_date
- date
- distance(m)
- field
- field_name
- number of samples

#### Normalize Index Column Values

##### Visualize Leading & Trailing Whitespace

Having peeked ahead a bit, I've noticed irregular use of space characters in some of the non-numeric values within the `DataFrame` objects. I could just strip all leading and trailing spaces, but I might overlook something along the way, and it's not a good habit to attempt to clean text too early in the course of analysis.

If I could see the spaces more easily, I could be more acutely aware of their existence. To visualize these invisible characters, I'll compose a function that wraps any text value in square brackets and returns the result. Displaying values this way makes trailing or leading whitespace obvious, without overwriting any data.

In [ ]:
def wrap_brackets(x):
    return x if pandas.isna(x) else f'[{str(x)}]'


# Example:
pandas.Series([
    pandas.np.NaN,
    ' a ',
    'b',
    ' ',
    '',
]).apply(wrap_brackets)

I'll reuse this function later.

##### Date

The columns containing date information were normalized in an [earlier stage] of analysis. Those columns are ready to align in combination with other unique index labels.

[earlier stage]: #Convert-Date-&amp;amp;-Time-Format

##### Crop

Here are the `crop` field values for both data frames. (Since I've noticed some sneaky whitespace, I'll wrap the `crop` values in brackets for visualization.)

In [ ]:
crops = pandas.concat(
    (
        frame.crop.apply(str)
        for frame in compare_sheets.values()
    ),
    keys=sheet_names,
    sort=True,
).drop_duplicates().reset_index(drop=True).sort_values()
crops.apply(wrap_brackets)

Clearly, there are some variations that should be corrected, in both data frames.

- whitespace
- letter case
- word separation

I'll write a function that transforms any given "crop" value into a uniform representation of the crop it's intended to represent.

In [ ]:
def normalize_str(value, separator=' '):
    if value is pandas.np.nan:
        return value

    str_value = str(value)

    # Add separators between words, title case
    is_mixed_case = str_value.upper() != str_value.lower() and not (str_value.islower() or str_value.isupper())
    if is_mixed_case:
        word_index = [
            index for index, char in enumerate(str_value) 
            if char.isupper()
        ] + [None]
        if word_index:
            words = [
                str_value[word_index[i]:word_index[i + 1]].strip()
                for i in range(len(word_index) - 1)
            ]
            str_value = separator.join(words)
    
    transformed = re.compile(r'[^a-zA-Z0-9]').sub(separator, str(str_value).title())
    
    # De-pluralize
    if transformed.endswith('Oats'):
        transformed = transformed[:-1]
    
    return transformed

Previewing the results, with the square brackets added again:

In [ ]:
pandas.concat(
    (
        crops,
        crops.apply(normalize_str),
    ),
    keys=("Before", "After"),
    axis='columns',
).sort_values('After').applymap(wrap_brackets)

There are some concerning names, such as "nan", "unlisted", and "0", but the renaming result looks good. I'll apply the changes:

In [ ]:
for frame in (hc, s2):
    frame.crop = frame.crop.apply(normalize_str)

##### Site

In [ ]:
site_values = (
    pandas.concat(
        (frame[['site']] for frame in (hc, s2)),
        keys=sheet_names,
        names=['Sheet Name', 'index',],
    )
    .drop_duplicates()
    .sort_values('site')
)
site_values.site.apply(wrap_brackets)

This is going to need some normalization. I'll write a string reducer function to transform any given value into a uniformly reducible representation by stripping insignificant characters and letter case. From that, I can build a hash table that maps to preferred representations, and apply that mapping to the values in a renaming operation.

###### Index by Reduced Label Value

In order to match badly formed labels with their normal representation, I need a string function similar to a hash, so the values in the data frame can be used to look up the preferred, normal form.

- convert numeric values to text
- strip non alphanumeric characters
- convert alphabetic characters to lower case

In [ ]:
def hash_like(value):
    return re.compile(r"[^a-z0-9]").sub('', str(value).lower())

Applying this to all frames:

In [ ]:
for frame in (hc, s2) + (site_values, ):
    frame['site_index'] = frame.site.apply(hash_like)
    frame.set_index('site_index', append=True, inplace=True)

###### Choosing Normal Form

If I had a larger data set, I could automatically find the most frequently used representation for any given reduced value ("hash"); I would use the `mode` function. Unfortunately, the groups are far too small and the values too varied:

In [ ]:
(
    site_values.site
    .apply(wrap_brackets)
    .unstack()
    .fillna('')
    .reset_index('index', drop=True)
)

Even if I strip the leading and trailing whitespace, I would still end up with ambiguous candidate selections. The work required to make a function that would know to how and when to convert labels like `Yellowcreek` to `Yellow Creek` would be unreasonable given the scope of this project, so automation might not be the best choice for choosing normal forms.

I'll make the preferred identifier list manually, then apply the changes automatically. Here it is as a data frame with the appropriately reduced version of each label as the index:

In [ ]:
preferred_site_id = pandas.Series(
    name='site',
    data={hash_like(item): item
          for item in [
              'Alvena',
              'Clavet',
              'Indian Head',
              'Kernan',
              'Llewellyn',
              'Meadow Lake',
              'Melfort',
              'Outlook',
              'SEF',
              'Wakaw',
              'Yellow Creek',
          ]},
)
preferred_site_id.index.set_names(['site_index'], inplace=True)
preferred_site_id.to_frame().T

With this list, I can compare the reduced ("hashed") values to the ones in the actual data and apply the preferred name where it matches.

Everything that isn't in the preferred site name list:

In [ ]:
site_values[
    ~ site_values.index.get_level_values('site_index')
    .isin(preferred_site_id.index)
]

These outliers are from the unmatchable records I mentioned upon my first look at the [unique site] labels. I can move on without doing anything further on these.

[unique site]: #Outliers

Report on which records would be changed if I relabel `site`:

In [ ]:
pandas.concat(
    (
        site_values,
        preferred_site_id.to_frame().combine_first(site_values),
    ),
    keys=['Before', 'After'],
    axis='columns',
).reset_index('site_index', drop=True).applymap(wrap_brackets)

This looks good to me. I'll apply the names:

In [ ]:
for frame in (hc, s2):
    frame.loc[:, 'site'] = (
        preferred_site_id.to_frame()
        .combine_first(frame)
        .loc[:, 'site']
    )
    frame.reset_index(
        level='site_index',
        drop=True,
        inplace=True,
    )

I'll review the label values for `site` in both data frames, to see the result of those changes:

In [ ]:
(
    pandas.concat(
        (frame[['site']] for frame in (hc, s2)),
    )
    .loc[:, 'site']
    .reset_index(drop=True)
    .sort_values()
    .drop_duplicates()
    .apply(wrap_brackets)
)

##### Field

The columns **hc**.***field*** and **s2**.***field_name*** seem to relate to ***site*** and ***crop***. Before I address the values of the columns, I'll rename **s2**.***field_name*** for consistency.

In [ ]:
for frame in (hc, s2):
    frame.rename(
        columns={'field_name': 'field'},
        inplace=True,
    )

Now, regarding the values, I suspect redundancy. Here's why I feel that way; look at some of the values from both data sets:

In [ ]:
index_column_names = ['site', 'crop', 'field']
pandas.concat(
    (
        hc[index_column_names],
        s2[index_column_names],
    ),
    keys=sheet_names,
#     names=['worksheet', 'index',],
).applymap(wrap_brackets).head(15)

It's clear to my eyes that in most cases, `site` and `crop` have been concatenated together to produce the value in `field` or `field_name`.

- site + crop = field(_name)

Primary data:

- site
- crop

Aggregated data:

- field
- field_name

Therefore, in those cases, I can safely disregard those derived columns and rely on the the more normalized forms for indexing. That is to say I think it's most beneficial to use `crop` and `site` whenever possible.

In [ ]:
(
    pandas.merge(
        s2.reset_index(), hc.reset_index(),
        suffixes=('_s2', '_hc'),
        how='inner',
        on=['date', 'crop', 'site', ],
    )
    .set_index(['date', 'crop', 'site',])
    .sort_index(level=['date', 'crop', 'site',])
    .loc[:, ['field_s2', 'field_hc']]
    .stack()
    .reset_index(level=-1, drop=True)
    .reset_index()
    .drop_duplicates()
    .rename(columns={0: 'field'})
#     .set_index(['date', 'crop', 'site',])
)

#### Number of Samples & Distance

The ***number of samples*** field is only present in **Sheet2**. I want to know a little bit about the values:

In [ ]:
s2['number of samples'].value_counts(dropna=False)

There are many missing values in this column. I presume the number of samples relates to the "combined" values, but I need to confirm:

In [ ]:
s2.loc[
    (
        s2['number of samples'].isna()
    ) | (
        ~ s2['distance(m)'].apply(
            isinstance,
            args=((int, float,),)
        )
    ),
    ['distance(m)', 'number of samples']
].drop_duplicates()

This confirms that where the ***number of samples*** is not indicated, the corresponding ***distance(m)*** value is numeric. The converse is also true: non-numeric ***distance(m)*** corresponds with positive values in ***number of samples***. That's consistent with my expectations for the appearance of labels on previously aggregated data.

### Observation Data

Now that the data frame indices I built from the `date`, `site`, `field` and `crop` columns align, I need to determine which observations were recorded in both data frames.

##### @todo:

- [ ] move code block to appropriate place, annotate

In [ ]:
for frame in (hc, s2):
    frame.set_index(index_column_names, inplace=True)

##### @todo:

- [ ] select aphid columns, check if others exist in common

### Compare Group Sums to Pre-existing "Combined"

I didn't expect that the data set from `Sheet2` would have sums (presumably from groupings of `distance(m)` values), yet also some non-aggregated values. Normally, these wouldn't be mixed, because they represent different dimensional orders. Because this data set's dimensional order is inconsistent, there is probably redundancy in the total information available, and possibly contradictions.

Any redundancy, whether contradictory or not, would affect calculation for the intended [objective], unless there's exactly one record for each space and time combination — that is to say, if there are discrete values as well as previously "combined" values for the same point along the index, I'll have to avoid including the pre-calculated sum when aggregating my own sums, otherwise the resulting totals will be doubled.

[objective]: #Objective

I'll separate discrete and aggregated sets, then compare my sums of discrete sample values to pre-existing aggregations.

Before I can compare